In [104]:
from dataclasses import dataclass
from typing import Mapping, Dict, Tuple, TypeVar, Callable, Iterator, Sequence
import sys 
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath("/Users/justincramer/Documents/Coding/CME241/RL-book/"))
import rl.monte_carlo as mc
from rl.chapter3.simple_inventory_mdp_cap import InventoryState, SimpleInventoryMDPCap
from rl.approximate_dynamic_programming import (ValueFunctionApprox,
                                                QValueFunctionApprox,
                                                NTStateDistribution)
from rl.dynamic_programming import policy_iteration_result
from rl.distribution import Distribution, Choose, Gaussian
from rl.function_approx import DNNSpec, AdamGradient, DNNApprox
from rl.approximate_dynamic_programming import back_opt_vf_and_policy, \
    back_opt_qvf, ValueFunctionApprox, QValueFunctionApprox

from rl.iterate import last
from rl.markov_decision_process import MarkovDecisionProcess, FiniteMarkovDecisionProcess, Policy, \
    TransitionStep, NonTerminal
from rl.policy import DeterministicPolicy, RandomPolicy, UniformPolicy, FiniteDeterministicPolicy
import rl.markov_process as mp
from rl.returns import returns
from rl.function_approx import FunctionApprox, Tabular
from rl.chapter7.asset_alloc_discrete import AssetAllocDiscrete
import rl.td as td
import itertools
from pprint import pprint
S = TypeVar('S')
A = TypeVar('A')

## 2.

#### Optimal VF and Policy for Simple Inventory MDP

In [53]:
# Simple inventory MDP
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9
si_mdp: FiniteMarkovDecisionProcess[InventoryState, int] =\
        SimpleInventoryMDPCap(
            capacity=user_capacity,
            poisson_lambda=user_poisson_lambda,
            holding_cost=user_holding_cost,
            stockout_cost=user_stockout_cost
        )
    
opt_vf_pi, opt_policy_pi = policy_iteration_result(
    si_mdp,
    gamma=user_gamma
)
    
pprint(opt_vf_pi)
print(opt_policy_pi)

{NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.991900091403522,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991900091403522,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.660960231637496,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.991900091403522,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.89485578163003,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.660960231637496}
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



#### Optimal VF and Policy for Asset Allocation

In [183]:
steps: int = 4
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_stdev: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(
    2 / 3 * base_alloc,
    4 / 3 * base_alloc,
    11
)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = \
    [
        lambda _: 1.,
        lambda w_x: w_x[0],
        lambda w_x: w_x[1],
        lambda w_x: w_x[1] * w_x[1]
    ]
dnn: DNNSpec = DNNSpec(
    neurons=[],
    bias=False,
    hidden_activation=lambda x: x,
    hidden_activation_deriv=lambda y: np.ones_like(y),
    output_activation=lambda x: - np.sign(a) * np.exp(-x),
    output_activation_deriv=lambda y: -y
)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_stdev)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
    risky_return_distributions=risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr
)

it_qvf: Iterator[QValueFunctionApprox[float, float]] = \
        aad.backward_induction_qvf()

it_qvf: Iterator[QValueFunctionApprox[float, float]] = \
        aad.backward_induction_qvf()

print("Backward Induction on Q-Value Function")
print("--------------------------------------")
print()
for t, q in enumerate(it_qvf):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = max(
        ((q((NonTerminal(init_wealth), ac)), ac) for ac in alloc_choices),
        key=itemgetter(0)
    )[1]
    val: float = max(q((NonTerminal(init_wealth), ac))
                     for ac in alloc_choices)
    print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
    print("Optimal Weights below:")
    for wts in q.weights:
        pprint(wts.weights)
    print()


KeyboardInterrupt: 

### 1.

In [93]:
# Utility to get optimal value function and policy for optimal Q-value function
def get_opt_val_policy(mdp, qvf):
    
    states = mdp.non_terminal_states
    values = {}
    policy_dict = {}
    for s in states:
        _, a = qvf.argmax((s, a) for a in mdp.actions(s))
        val = qvf((s, a))
        values[s] = val
        policy_dict[s] = a
    return values, FiniteDeterministicPolicy(policy_dict)
        

#### Simple Inventory MDP

In [178]:
# Uniform distribution over non-terminal states
init_distr: NTStateDistribution[InventoryState] = Choose(si_mdp.non_terminal_states)
approx_0: QValueFunctionApprox[InventoryState, int] = Tabular()
num_episodes: int = 1000
# Q-Value Function Approximation
qvf = last(itertools.islice(mc.glie_mc_control(si_mdp, init_distr, approx_0, user_gamma, lambda k: 1 / k), 
                            num_episodes))   

In [179]:
get_opt_val_policy(si_mdp, qvf)

({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.62597350682885,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -28.054374978569978,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.46623407438349,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -29.021325006751088,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.436113768855254,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.423106126098315},
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=0)): Do Action 2
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=1)): Do Action 1
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=2)): Do Action 0
 For State NonTerminal(state=InventoryState(on_hand=1, on_order=0)): Do Action 1
 For State NonTerminal(state=InventoryState(on_hand=1, on_order=1)): Do Action 0
 For State NonTerminal(state=InventoryState(on_hand=2, on_order=0)): Do Action 0)

#### Asset Allocation

In [225]:
mdp = aad.get_mdp(0)
init_distr = init_wealth_distr
approx_0 = aad.get_qvf_func_approx()

In [233]:
num_episodes: int = 1000
# Q-Value Function Approximation
qvf = last(itertools.islice(mc.glie_mc_control(mdp, init_distr, approx_0, user_gamma, lambda k: 1 / k), 
                            num_episodes))   

In [258]:
next(mc.glie_mc_control(mdp, init_distr, approx_0, user_gamma, lambda k: 1 / k))

DNNApprox(feature_functions=[<function AssetAllocDiscrete.get_qvf_func_approx.<locals>.this_f at 0x7fa5e8106e50>, <function AssetAllocDiscrete.get_qvf_func_approx.<locals>.this_f at 0x7fa5e8106f70>, <function AssetAllocDiscrete.get_qvf_func_approx.<locals>.this_f at 0x7fa5e8103040>, <function AssetAllocDiscrete.get_qvf_func_approx.<locals>.this_f at 0x7fa5e81030d0>], dnn_spec=DNNSpec(neurons=[], bias=False, hidden_activation=<function <lambda> at 0x7fa655cea790>, hidden_activation_deriv=<function <lambda> at 0x7fa655cea820>, output_activation=<function <lambda> at 0x7fa655cea700>, output_activation_deriv=<function <lambda> at 0x7fa655cea0d0>), regularization_coeff=0.0, weights=[Weights(adam_gradient=AdamGradient(learning_rate=0.1, decay1=0.9, decay2=0.999), time=0, weights=array([[-0.86178018, -1.11402238,  0.11125622,  1.01734087]]), adam_cache1=array([[0., 0., 0., 0.]]), adam_cache2=array([[0., 0., 0., 0.]]))])

### 2.

#### Simple Inventory MDP

In [182]:
# Uniform distribution over non-terminal states
init_distr: NTStateDistribution[InventoryState] = Choose(si_mdp.non_terminal_states)
approx_0: QValueFunctionApprox[InventoryState, int] = Tabular()
num_episodes: int = 10000
max_episodes_length: int = 100
qvf = last(itertools.islice(td.q_learning(si_mdp, mc.epsilon_greedy_policy, init_distr, approx_0, user_gamma, 
                                          max_episodes_length), 
                            num_episodes))
get_opt_val_policy(si_mdp, qvf)

qvf = last(itertools.islice(td.glie_sarsa(si_mdp, init_distr, approx_0, user_gamma, lambda k: 1 / k,
                                          max_episodes_length),
                            num_episodes))
get_opt_val_policy(si_mdp, qvf)



({NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -24.310699840956065,
  NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -17.257659229168535,
  NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -17.487758758963512,
  NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -18.218688806408593,
  NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -18.50992058741949,
  NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -19.68121790577847},
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=0)): Do Action 1
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=1)): Do Action 1
 For State NonTerminal(state=InventoryState(on_hand=0, on_order=2)): Do Action 0
 For State NonTerminal(state=InventoryState(on_hand=1, on_order=0)): Do Action 1
 For State NonTerminal(state=InventoryState(on_hand=1, on_order=1)): Do Action 0
 For State NonTerminal(state=InventoryState(on_hand=2, on_order=0)): Do Action 0)